In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from prophet import Prophet
import plotly.graph_objects as go

C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
usdInr = pd.read_csv(r'C:/Users/uzmap/Documents/GitHub/ForEx/USDINR/USDINR.csv')
usdInr.drop('Unnamed: 0', axis=1, inplace=True)
usdInr.head()

,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Deseasonalized_Day_high,Deseasonalized_Day_low,Deseasonalized_Open_price,EMA_50,EMA_100,EMA_200,RSI,MACD,Signal_Line,MACD_Histogram,SMA,Upper Band,Lower Band
0,2014-11-07,61.390,61.620,61.340,61.400,USD/INR,61.609403,61.339012,61.390960,61.400000,61.400000,61.400000,0.0,0.000000,0.000000,0.000000,61.7496,62.164424,61.334776
1,2014-11-10,61.500,61.635,61.340,61.495,USD/INR,61.625872,61.330301,61.492726,61.403725,61.401881,61.400945,0.0,0.007578,0.001516,0.006063,61.7496,62.164424,61.334776
2,2014-11-11,61.530,61.555,61.505,61.508,USD/INR,61.546894,61.500125,61.514536,61.407815,61.403983,61.402010,0.0,0.014466,0.004106,0.010361,61.7496,62.164424,61.334776
3,2014-11-12,61.508,61.560,61.350,61.391,USD/INR,61.568238,61.358593,61.504599,61.407155,61.403725,61.401901,0.0,0.010365,0.005358,0.005007,61.7496,62.164424,61.334776
4,2014-11-13,61.368,61.623,61.350,61.565,USD/INR,61.636544,61.351053,61.386236,61.413345,61.406919,61.403524,0.0,0.020914,0.008469,0.012445,61.7496,62.164424,61.334776


In [3]:
import numpy as np

def evaluate_trading_metrics(y_true, y_pred, initial_capital=1000):
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    # Ensure lengths match
    min_len = min(len(y_true), len(y_pred))
    y_true = y_true[:min_len]
    y_pred = y_pred[:min_len]

    # Compute directional accuracy
    true_diff = np.sign(np.diff(y_true))
    pred_diff = np.sign(np.diff(y_pred))
    correct = np.sum(true_diff == pred_diff)
    trading_accuracy = correct / len(true_diff) * 100

    # Simulate trading strategy
    capital = [initial_capital]
    for i in range(1, len(y_pred)):
        # If model says price will go up -> buy (position = +1)
        # If model says price will go down -> short (position = -1)
        position = 1 if y_pred[i] > y_pred[i - 1] else -1
        profit = position * (y_true[i] - y_true[i - 1])
        capital.append(capital[-1] + profit)

    capital = np.array(capital)
    returns = np.diff(capital) / capital[:-1]

    # ROI
    roi = (capital[-1] - initial_capital) / initial_capital * 100

    # Sharpe Ratio (assuming risk-free rate is 0)
    sharpe_ratio = np.mean(returns) / np.std(returns) if np.std(returns) > 0 else 0

    # Max Drawdown
    peak = np.maximum.accumulate(capital)
    drawdown = (peak - capital) / peak
    max_drawdown = np.max(drawdown) * 100  # as a percentage

    return {
        "Trading Accuracy (%)": trading_accuracy,
        "ROI (%)": roi,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown (%)": max_drawdown
    }

In [4]:
def directional_accuracy(actual, predicted):
    # Ensure 1D arrays
    actual = np.ravel(actual)
    predicted = np.ravel(predicted)

    # Compute direction change
    actual_diff = np.diff(actual)
    pred_diff = np.diff(predicted)

    # Compare directions
    correct_direction = np.sign(actual_diff) == np.sign(pred_diff)

    return np.mean(correct_direction) * 100

In [5]:
# Calculate MAE, MSE, RMSE, and MAPE for the train and test sets
def evaluate_model(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    accuracy = 100 - mape  # Accuracy-like score as (100% - MAPE)
    
    return mae, mse, rmse, mape, accuracy

In [6]:
# Load the data
df = usdInr
df.rename(columns={'Date': 'ds', 'Closing_price': 'y'}, inplace=True)
df.dropna(inplace=True)

# Add the 'cap' column for logistic growth (set it to the max of 'y')
df['cap'] = df['y'].max()

# Split the data into training and testing sets
train_size = int(0.8 * len(df))
train = df[:train_size]
test = df[train_size:]

# Set up the hyperparameters grid
param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1],
    'seasonality_prior_scale': [1, 10, 20],
    'holidays_prior_scale': [10, 15, 20],
    'changepoint_range': [0.8, 0.9],
    'growth': ['linear', 'logistic']  # Will test both linear and logistic growth
}

# Initialize variables to store the best model and best score
best_model = None
best_score = float('inf')
best_params = {}
best_metrics = {}
best_trading_metrics = {}

# Grid Search Loop
for changepoint_prior_scale in param_grid['changepoint_prior_scale']:
    for seasonality_prior_scale in param_grid['seasonality_prior_scale']:
        for holidays_prior_scale in param_grid['holidays_prior_scale']:
            for changepoint_range in param_grid['changepoint_range']:
                for growth in param_grid['growth']:
                    
                    # Initialize Prophet with the current set of hyperparameters
                    prophet_model = Prophet(
                        changepoint_prior_scale=changepoint_prior_scale,
                        seasonality_prior_scale=seasonality_prior_scale,
                        holidays_prior_scale=holidays_prior_scale,
                        changepoint_range=changepoint_range,
                        growth=growth
                    )
                    
                    # Fit the model (with logistic growth if needed)
                    if growth == 'logistic':
                        prophet_model.fit(train[['ds', 'y', 'cap']])
                    else:
                        prophet_model.fit(train[['ds', 'y']])
                    
                    # Create future dataframe for predictions
                    future = prophet_model.make_future_dataframe(periods=len(test))
                    
                    # Add the 'cap' column for logistic growth
                    if growth == 'logistic':
                        future['cap'] = df['cap'].max()
                    
                    forecast = prophet_model.predict(future)
                    
                    # Evaluate model performance on the test set
                    test_yhat = forecast['yhat'][-len(test):].values
                    mae, mse, rmse, mape, accuracy = evaluate_model(test['y'].values, test_yhat)
                    
                    # Calculate additional metrics
                    trading_metrics = evaluate_trading_metrics(test['y'].values, test_yhat)
                    dir_acc = directional_accuracy(test['y'].values, test_yhat)
                    
                    # If the model has better performance, store it
                    if mse < best_score:
                        best_score = mse
                        best_model = prophet_model
                        best_params = {
                            'changepoint_prior_scale': changepoint_prior_scale,
                            'seasonality_prior_scale': seasonality_prior_scale,
                            'holidays_prior_scale': holidays_prior_scale,
                            'changepoint_range': changepoint_range,
                            'growth': growth
                        }
                        # Save metrics for the best model
                        best_metrics = {
                            'MAE': mae,
                            'MSE': mse,
                            'RMSE': rmse,
                            'MAPE': mape,
                            'Accuracy': accuracy,
                            'Directional Accuracy': dir_acc
                        }
                        best_trading_metrics = trading_metrics

# Output the best hyperparameters and score
print("Best Hyperparameters:", best_params)
print("Best Model Metrics:")
for metric, value in best_metrics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
print("\nTrading Metrics for Best Model:")
print(best_trading_metrics)

# Recalculate final metrics for consistency (optional)
future_best = best_model.make_future_dataframe(periods=len(test))
if best_params['growth'] == 'logistic':
    future_best['cap'] = df['cap'].max()

best_forecast = best_model.predict(future_best)
test_yhat = best_forecast['yhat'][-len(test):].values
mae, mse, rmse, mape, accuracy = evaluate_model(test['y'].values, test_yhat)
dir_acc = directional_accuracy(test['y'].values, test_yhat)
trading_metrics = evaluate_trading_metrics(test['y'].values, test_yhat)

print("\nFinal Evaluation on Test Set:")
print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.2f}%, Accuracy: {accuracy:.2f}%, Directional Accuracy%: {dir_acc:.2f}")
print("\nTrading Metrics:")
print(trading_metrics)

20:28:14 - cmdstanpy - INFO - Chain [1] start processing
20:28:15 - cmdstanpy - INFO - Chain [1] done processing
20:28:15 - cmdstanpy - INFO - Chain [1] start processing
20:28:16 - cmdstanpy - INFO - Chain [1] done processing
20:28:20 - cmdstanpy - INFO - Chain [1] start processing
20:28:21 - cmdstanpy - INFO - Chain [1] done processing
20:28:21 - cmdstanpy - INFO - Chain [1] start processing
20:28:22 - cmdstanpy - INFO - Chain [1] done processing
20:28:26 - cmdstanpy - INFO - Chain [1] start processing
20:28:27 - cmdstanpy - INFO - Chain [1] done processing
20:28:27 - cmdstanpy - INFO - Chain [1] start processing
20:28:28 - cmdstanpy - INFO - Chain [1] done processing
20:28:32 - cmdstanpy - INFO - Chain [1] start processing
20:28:32 - cmdstanpy - INFO - Chain [1] done processing
20:28:33 - cmdstanpy - INFO - Chain [1] start processing
20:28:33 - cmdstanpy - INFO - Chain [1] done processing
20:28:37 - cmdstanpy - INFO - Chain [1] start processing
20:28:38 - cmdstanpy - INFO - Chain [1]

Best Hyperparameters: {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 1, 'holidays_prior_scale': 10, 'changepoint_range': 0.8, 'growth': 'linear'}
Best Model Metrics:
MAE: 3.4841
MSE: 14.5553
RMSE: 3.8151
MAPE: 4.1526
Accuracy: 95.8474
Directional Accuracy: 53.9741

Trading Metrics for Best Model:
{'Trading Accuracy (%)': 53.97412199630314, 'ROI (%)': 0.5310999999999695, 'Sharpe Ratio': 0.07176276964570812, 'Max Drawdown (%)': 0.20370642449561166}

Final Evaluation on Test Set:
MAE: 3.4841, MSE: 14.5553, RMSE: 3.8151, MAPE: 4.15%, Accuracy: 95.85%, Directional Accuracy%: 53.97

Trading Metrics:
{'Trading Accuracy (%)': 53.97412199630314, 'ROI (%)': 0.5310999999999695, 'Sharpe Ratio': 0.07176276964570812, 'Max Drawdown (%)': 0.20370642449561166}


In [7]:
import pandas as pd
import numpy as np
from prophet import Prophet
import plotly.graph_objects as go

# -------------------------------
# ✅ 1. Prepare Data
# -------------------------------
df.rename(columns={'Date': 'ds', 'Closing_price': 'y'}, inplace=True)
df.dropna(inplace=True)

# List of additional regressors
regressors = [
    'Deseasonalized_Day_high', 'Deseasonalized_Day_low',
    'Deseasonalized_Open_price', 'EMA_100', 'EMA_200', 'EMA_50', 'RSI',
    'MACD', 'Signal_Line', 'MACD_Histogram', 'SMA', 'Upper Band', 'Lower Band'
]

# Check regressors
missing_regressors = [r for r in regressors if r not in df.columns]
if missing_regressors:
    raise ValueError(f"Missing regressors in dataset: {missing_regressors}")

# -------------------------------
# ✅ 2. Train-Test Split (Sequential 85-15)
# -------------------------------
split_idx = int(len(df) * 0.85)
train = df.iloc[:split_idx].copy()
test = df.iloc[split_idx:].copy()

# -------------------------------
# ✅ 4. Train Prophet Model
# -------------------------------
best_params = {
    'changepoint_prior_scale': 0.05,
    'seasonality_prior_scale': 10,
    'holidays_prior_scale': 10,
    'changepoint_range': 0.9,
    'growth': 'linear'
}

model = Prophet(
    changepoint_prior_scale=best_params['changepoint_prior_scale'],
    seasonality_prior_scale=best_params['seasonality_prior_scale'],
    holidays_prior_scale=best_params['holidays_prior_scale'],
    changepoint_range=best_params['changepoint_range'],
    growth=best_params['growth']
)

# Add regressors
for reg in regressors:
    model.add_regressor(reg)

# Fit model on training data
model.fit(train[['ds', 'y'] + regressors])

# -------------------------------
# ✅ 5. Predict for Full Dataset & Slice Test
# -------------------------------
future = df[['ds'] + regressors].copy()
forecast = model.predict(future)

# Slice forecast for test period
test_forecast = forecast[forecast['ds'].isin(test['ds'])]
test_yhat = test_forecast['yhat'].values

# -------------------------------
# ✅ 6. Metrics
# -------------------------------
test_metrics = evaluate_model(test['y'].values, test_yhat)
dir_acc = directional_accuracy(test['y'].values, test_yhat)
trading_metrics = evaluate_trading_metrics(test['y'].values, test_yhat)

# Print metrics
print("\nTest Metrics:")
print(f"MAE: {test_metrics[0]:.4f}, MSE: {test_metrics[1]:.4f}, RMSE: {test_metrics[2]:.4f}, MAPE: {test_metrics[3]:.2f}%, Accuracy: {test_metrics[4]:.2f}%")
print(f"Directional Accuracy: {dir_acc:.2f}%")
print("\nTrading Metrics:")
print(trading_metrics)

# -------------------------------
# ✅ 7. Plot Test Predictions vs Actual
# -------------------------------
fig = go.Figure()

# Actual test values
fig.add_trace(go.Scatter(x=test['ds'], y=test['y'],
                         mode='lines', name='Actual', line=dict(color='blue')))

# Predicted values
fig.add_trace(go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat'],
                         mode='lines', name='Predicted', line=dict(color='red')))

# Confidence intervals
fig.add_trace(go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat_upper'],
                         mode='lines', name='Upper Bound', line=dict(color='lightgrey', dash='dot')))
fig.add_trace(go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat_lower'],
                         mode='lines', name='Lower Bound', line=dict(color='lightgrey', dash='dot')))

fig.update_layout(
    title="Prophet Test Set Predictions vs Actual",
    xaxis_title="Date",
    yaxis_title="Closing Price",
    template="plotly_white"
)
fig.show()

20:35:01 - cmdstanpy - INFO - Chain [1] start processing
20:35:02 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\uzmap\AppData\Local\Temp\ipykernel_10320\2192487926.py:64: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_forecast = forecast[forecast['ds'].isin(test['ds'])]



Test Metrics:
MAE: 0.0406, MSE: 0.0031, RMSE: 0.0556, MAPE: 0.05%, Accuracy: 99.95%
Directional Accuracy: 79.56%

Trading Metrics:
{'Trading Accuracy (%)': 79.55665024630541, 'ROI (%)': 2.8762999999999237, 'Sharpe Ratio': 0.7238668109896128, 'Max Drawdown (%)': 0.01835722376285025}
